In [1]:
import os

import ir_datasets
import evaluate
from tqdm.notebook import tqdm

from irise import INDEX_DIR
from irise.indexer import Indexer
from irise.pipeline import SearchPipeline

2024-06-05 22:02:44.984058: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-05 22:02:45.597749: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:
SYSTEM = "TFIDF"
dataset = ir_datasets.load("beir/msmarco/test")
pipe = SearchPipeline(index_path=INDEX_DIR / "irise_index_advanced")

In [31]:
predictions = {
    "query": [],
    "q0": [],
    "docid": [],
    "score": [],
    "rank": [],
    "system": [],
}

for query in tqdm(dataset.queries_iter(), total=dataset.queries_count()):
    results = pipe(query.text, return_dict=False)
    if not results:
        predictions["query"].append(int(query.query_id))
        predictions["q0"].append("q0")
        predictions["docid"].append(str(-1))
        predictions["score"].append(-1)
        predictions["rank"].append(-1)
        predictions["system"].append(SYSTEM)
    else:
        for rank, (doc_id, score) in enumerate(results):
            predictions["query"].append(int(query.query_id))
            predictions["q0"].append("q")
            predictions["docid"].append(doc_id)
            predictions["score"].append(score)
            predictions["rank"].append(rank)
            predictions["system"].append(SYSTEM)

  0%|          | 0/43 [00:00<?, ?it/s]

In [23]:
qrels = {
    "query": [],
    "q0": [],
    "docid": [],
    "rel": [],
}
for qrel in tqdm(dataset.qrels_iter(), total=dataset.qrels_count()):
    qrels["query"].append(int(qrel.query_id))
    qrels["q0"].append("q0")
    qrels["docid"].append(qrel.doc_id)
    qrels["rel"].append(qrel.relevance)

  0%|          | 0/9260 [00:00<?, ?it/s]

In [3]:
trec_eval = evaluate.load("trec_eval")

In [33]:
results_tfidf = trec_eval.compute(references=[qrels], predictions=[predictions])

/home/devrim/anaconda3/envs/irise/lib/python3.11/site-packages/trectools/trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()


In [73]:
results

{'runid': 'bm25',
 'num_ret': 193,
 'num_rel': 4102,
 'num_rel_ret': 91,
 'num_q': 43,
 'map': 0.026539430854207936,
 'gm_map': 0.0003330836456447731,
 'bpref': 0.031150645416553026,
 'Rprec': 0.03386240921870229,
 'recip_rank': 0.37984496124031014,
 'P@5': 0.21395348837209305,
 'P@10': 0.13488372093023254,
 'P@15': 0.11007751937984496,
 'P@20': 0.09418604651162793,
 'P@30': 0.07054263565891472,
 'P@100': 0.021162790697674416,
 'P@200': 0.010581395348837208,
 'P@500': 0.0042325581395348845,
 'P@1000': 0.0021162790697674422,
 'NDCG@5': 0.19136769935348993,
 'NDCG@10': 0.1422570019702889,
 'NDCG@15': 0.12542659497485611,
 'NDCG@20': 0.1147591332765549,
 'NDCG@30': 0.10008778327081985,
 'NDCG@100': 0.07321093743024241,
 'NDCG@200': 0.07117612916274589,
 'NDCG@500': 0.07113236503969173,
 'NDCG@1000': 0.07113236503969173}

In [84]:
results_tfidf

{'runid': 'TFIDF',
 'num_ret': 37,
 'num_rel': 296,
 'num_rel_ret': 20,
 'num_q': 4,
 'map': 0.043620841867515685,
 'gm_map': 0.0006929465911493908,
 'bpref': 0.05859653172075274,
 'Rprec': 0.06038970697605236,
 'recip_rank': 0.375,
 'P@5': 0.30000000000000004,
 'P@10': 0.22499999999999998,
 'P@15': 0.19999999999999998,
 'P@20': 0.1875,
 'P@30': 0.16666666666666666,
 'P@100': 0.049999999999999996,
 'P@200': 0.024999999999999998,
 'P@500': 0.009999999999999998,
 'P@1000': 0.004999999999999999,
 'NDCG@5': 0.2012167986481089,
 'NDCG@10': 0.15548921332871304,
 'NDCG@15': 0.13850637345559375,
 'NDCG@20': 0.14108855843244605,
 'NDCG@30': 0.12411529832289284,
 'NDCG@100': 0.08639438422761744,
 'NDCG@200': 0.08639438422761744,
 'NDCG@500': 0.08639438422761744,
 'NDCG@1000': 0.08639438422761744}

In [34]:
results_tfidf

{'runid': 'TFIDF',
 'num_ret': 430,
 'num_rel': 4102,
 'num_rel_ret': 353,
 'num_q': 43,
 'map': 0.14863877029136405,
 'gm_map': 0.10623112713697273,
 'bpref': 0.15655697050989456,
 'Rprec': 0.15928905616616532,
 'recip_rank': 0.9593023255813954,
 'P@5': 0.8790697674418604,
 'P@10': 0.8209302325581395,
 'P@15': 0.5472868217054264,
 'P@20': 0.41046511627906973,
 'P@30': 0.2736434108527132,
 'P@100': 0.08209302325581395,
 'P@200': 0.04104651162790698,
 'P@500': 0.01641860465116279,
 'P@1000': 0.008209302325581395,
 'NDCG@5': 0.729520788480906,
 'NDCG@10': 0.7064472702743727,
 'NDCG@15': 0.5765216412900166,
 'NDCG@20': 0.5030617383987054,
 'NDCG@30': 0.42685434713660225,
 'NDCG@100': 0.3201343402421015,
 'NDCG@200': 0.30607353920002395,
 'NDCG@500': 0.30394912747731856,
 'NDCG@1000': 0.30394912747731856}